# Immitation learning

## Data

### Imports

In [ ]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
from PIL import Image

from io import BytesIO
import os

In [ ]:
with ZipFile('recording_0.zip', 'r') as zf:
    zf.extractall()

In [ ]:
tab_df = pd.read_csv('tabular.csv')

In [ ]:
img_df = []

images_folder = 'images'
# Due to memory problems, we can only read 5000 images at a time
images = os.listdir(images_folder)[:5000]

for index, file in enumerate(images):
    img_df.append(np.array(Image.open(f'{images_folder}/{file}'), dtype=np.uint8))
    
    completion = 100*(index+1) / len(images)
    print(f'{completion:.2f}%', end='\r')
print()

img_df = np.array(img_df, dtype=np.uint8)

In [ ]:
columns = []

for i in range(img_df.shape[1] * img_df.shape[2]):
    columns.extend([f'r{i}', f'g{i}', f'b{i}'])
    
print(len(columns))

In [ ]:
img_df = pd.DataFrame(img_df.reshape((img_df.shape[0], img_df.shape[1]*img_df.shape[2]*img_df.shape[3]))
                      , columns=columns)

In [ ]:
img_df.shape

## Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D

model = Sequential()

model.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(640,800,3)))
model.add(MaxPooling2D())

model.add(Conv2D(64, kernel_size=5, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(2, activation='tanh'))

In [ ]:
from keras.utils import plot_model

plot_model(model, 'model.png', show_shapes=True)

In [ ]:
model.compile(loss='mean_absolute_error', optimizer='adam')